=== Test problem 2: Flow past a cylinder ===

idx{cylinder flow}

We now turn our attention to a more challenging problem: flow
past a circular cylinder. The geometry and parameters are taken from
problem DFG 2D-2 in the "FEATFLOW/1995-DFG benchmark suite":
"http://www.featflow.de/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html"
and is illustrated in Figure
ref{ftut1:navier_stokes_cylinder:geometry}. The kinematic viscosity is
given by $\nu = 0.001 = \mu/\varrho$ and the inflow velocity profile is
specified as

!bt
\[
  u(x, y, t) = \left(1.5 \cdot \frac{4y(0.41 - y)}{0.41^2}, 0\right),
\]
!et
which has a maximum magnitude of $1.5$ at $y = 0.41/2$. We do not
use any scaling for this problem since all exact parameters are known.

FIGURE:[fig/navier_stokes_cylinder_geometry, width=800 frac=0.95] Geometry for the flow past a cylinder test problem. Notice the slightly perturbed and unsymmetric geometry. label{ftut1:navier_stokes_cylinder:geometry}

=== FEniCS implementation ===

So far all our domains have been simple shapes such as a unit square or
a rectangular box. A number of such simple meshes may be created
using the built-in mesh classes in FEniCS
(`UnitIntervalMesh`,
`UnitSquareMesh`,
`UnitCubeMesh`,
`IntervalMesh`,
`RectangleMesh`,
`BoxMesh`).
FEniCS supports the creation of more complex meshes via a technique
called *constructive solid geometry* (CSG), which lets us define
geometries in terms of simple shapes (primitives) and set operations:
union, intersection, and set difference. The set operations are
encoded in FEniCS using the operators `+` (union), `*` (intersection),
and `-` (set difference). To access the CSG functionality in FEniCS,
one must import the FEniCS module `mshr` which provides the
extended meshing functionality of FEniCS.

idx{mshr}
idx{CSG}
idx{constructive solid geometry}

The geometry for the cylinder flow test problem can be defined easily
by first defining the rectangular channel and then subtracting the
circle:

!bc pycod
channel = Rectangle(Point(0, 0), Point(2.2, 0.41))
cylinder = Circle(Point(0.2, 0.2), 0.05)
domain = channel - cylinder
!ec
We may then create the mesh by calling the function `generate_mesh`:

!bc pycod
mesh = generate_mesh(domain, 64)
!ec
Here the argument `64` indicates that we want to resolve the geometry
with 64 cells across its diameter (the channel length).

idx{`generate_mesh`}

To solve the cylinder test problem, we only need to make a few minor
changes to the code we wrote for the channel flow test
case. Besides defining the new mesh, the only change we need to make
is to modify the boundary conditions and the time step size. The
boundaries are specified as follows:

!bc pycod
inflow   = 'near(x[0], 0)'
outflow  = 'near(x[0], 2.2)'
walls    = 'near(x[1], 0) || near(x[1], 0.41)'
cylinder = 'on_boundary && x[0]>0.1 && x[0]<0.3 && x[1]>0.1 && x[1]<0.3'
!ec
The last line may seem cryptic before you catch the idea: we want to pick
out all boundary points (`on_boundary`) that also lie within the 2D
domain $[0.1,0.3]\times [0.1,0.3]$, see Figure ref{ftut1:navier_stokes_cylinder:geometry}. The only possible points are then the points on the
circular boundary!

idx{`set_log_level`}
idx{`DEBUG` log level}
idx{`PROGRESS` log level}

In addition to these essential changes, we will make a number of small
changes to improve our solver. First, since we need to choose a
relatively small time step to compute the solution (a time step that
is too large will make the solution blow up) we add a progress bar so
that we can follow the progress of our computation. This can be done
as follows:

idx{`Progress`}

!bc pycod
progress = Progress('Time-stepping')
set_log_level(PROGRESS)

# Time-stepping
t = 0.0
for n in range(num_steps):

    # Update current time
    t += dt

    # Place computation here

    # Update progress bar
    progress.update(t / T)
!ec

!bnotice Log levels and printing in FEniCS
Notice the call to `set_log_level(PROGRESS)` which is essential to
make FEniCS actually display the progress bar. FEniCS is actually
quite informative about what is going on during a computation but the
amount of information printed to screen depends on the current log
level. Only messages with a priority higher than or equal to the
current log level will be displayed. The predefined log levels in
FEniCS are
`DBG`,
`TRACE`,
`PROGRESS`,
`INFO`,
`WARNING`,
`ERROR`, and
`CRITICAL`. By default, the log level is set to `INFO` which means
that messages at level `DBG`, `TRACE`, and `PROGRESS` will not be
printed. Users may print messages using the FEniCS functions `info`,
`warning`, and `error` which will print messages at the obvious log
level (and in the case of `error` also throw an exception and
exit). One may also use the call `log(level, message)` to print a
message at a specific log level.
!enotice

Since the system(s) of linear equations are significantly larger than
for the simple channel flow test problem, we choose to use an
iterative method instead of the default direct (sparse) solver used by
FEniCS when calling `solve`. Efficient solution of linear systems
arising from the discretization of PDEs requires the choice of both a
good iterative (Krylov subspace) method and a good
preconditioner. For this problem, we will simply use the biconjugate
gradient stabilized method (BiCGSTAB) and the conjugate gradient method. This can be done by adding the
keywords `bicgstab` or `cg` in the call to `solve`. We also specify
suitable preconditioners to speed up the computations:

idx{Krylov solver}
idx{preconditioner}

!bc pycod
solve(A1, u1.vector(), b1, 'bicgstab', 'hypre_amg')
solve(A2, p1.vector(), b2, 'bicgstab', 'hypre_amg')
solve(A3, u1.vector(), b3, 'cg', 'sor')
!ec

Finally, to be able to postprocess the computed solution in ParaView,
we store the solution to a file in each time step. We have previously
created files with the suffix `.pvd` for this purpose. In the example
program
"`${prog["heat_gaussian"]}.py`": "${src_url}/${prog["heat_gaussian"]}.py",
we first created a file named `heat_gaussian/solution.pvd` and then
saved the solution in each time step using
!bc pycod
vtkfile << (u, t)
!ec

For the present example, we will instead choose to save the solution
to XDMF format. This file format works similarly to the `.pvd` files
we have seen earlier but has several advantages. First, the storage is
much more efficient, both in terms of speed and file sizes. Second,
`.xdmf` files work in parallell, both for writing and reading
(postprocessing). Much like `.pvd` files, the actual data will not be
stored in the `.xdmf` file itself, but will instead be stored in a
(single) separate data file with the suffix `.hdf5` which is an
advanced file format designed for high-performance computing.
We create the XDMF files as follows:

!bc pycod
xdmffile_u = XDMFFile('navier_stokes_cylinder/velocity.xdmf')
xdmffile_p = XDMFFile('navier_stokes_cylinder/pressure.xdmf')
!ec
In each time step, we may then store the velocity and pressure by
!bc pycod
xdmffile_u.write(u, t)
xdmffile_p.write(p, t)
!ec

idx{`TimeSeries`}
idx{HDF5 format}
idx{XDMF format}
idx{`.hdf5` file}
idx{`.xdmf` file}

We also store the solution using a FEniCS `TimeSeries`. This allows us
to store the solution not for visualization, but for later reuse in a
computation as we will see in the next section. Using a `TimeSeries`
it is easy and efficient to read in solutions from certain points in
time during a simulation. The `TimeSeries` class also uses the HDF5
file format for efficient storage and access to data.

Figures ref{ftut1:fig:navier_stokes_cylinder:velocity} and
ref{ftut1:fig:navier_stokes_cylinder:pressure} show the velocity and
pressure at final time visualized in ParaView. For the visualization
of the velocity, we have used the _Glyph_ filter to visualize the
vector velocity field. For the visualization of the pressure, we have
used the _Warp By Scalar_ filter.

FIGURE:[fig/navier_stokes_cylinder_velocity, width=800 frac=0.95] Plot of the velocity for the cylinder test problem at final time. label{ftut1:fig:navier_stokes_cylinder:velocity}

FIGURE:[fig/navier_stokes_cylinder_pressure, width=800 frac=0.95] Plot of the pressure for the cylinder test problem at final time. label{ftut1:fig:navier_stokes_cylinder:pressure}

The complete code for the cylinder test problem looks as
follows:

@@@CODE vol1/python/navier_stokes_cylinder.py fromto: from fenics import@
This program can be found in the file "`${prog["navier_stokes_cylinder"]}.py`": "${src_url}/${prog["navier_stokes_cylinder"]}.py".
The reader should be advised that this example program is considerably
more demanding than our previous examples in terms of CPU time and
memory, but it should be possible to run the program on a reasonably
modern laptop.

idx{`${prog["navier_stokes_cylinder"]}.py`}

% if FORMAT not in ('latex', 'pdflatex'):
MOVIE: [mov/navier_stokes_cylinder.ogv, height=400,width=1000]
% endif